# Lab: Bring your own script with Amazon SageMaker

## Sklearn script mode training and serving
Script mode is a training script format for a number of supported frameworks that lets you execute the training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native SKlearn support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [Iris dataset](https://archive.ics.uci.edu/ml/datasets/iris). In this example, we will show how easily you can train a SageMaker using scikit-learn and with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker SKlearn container](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-docker-containers-scikit-learn-spark.html). 

## Set up the environment
Let's start by setting up the environment:

In [ ]:
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
import os
import tarfile
import pandas as pd

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
print(bucket)
#prefix = 'sagemaker/DEMO-BYO'

role = sagemaker.get_execution_role()

## Training data
we download the Iris data from UCI Machine Learning repository directly from the web.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

In [ ]:
data = pd.read_csv('iris.data', 
                   names=['sepal length', 'sepal width', 
                          'petal length', 'petal width', 
                          'label'])

data.to_csv('data.csv')


Load our data to our S3 ready for training using our script.

In [ ]:
import boto3

s3_session = boto3.Session().resource('s3')
s3_session.Bucket(bucket).Object('data/data.csv').upload_file('data.csv')


In [ ]:
!pygmentize 'main.py'


## Construct a script for brining your own SKlearn script to SageMaker
Your Scikit-learn training script must be a Python 3.6 compatible source file.

There is a 'main.py' file in the directory where you can create your script with some instructions in it.

Please take note of the following, when creating the script main.py to guide you through the process.

The training script is similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables. For example:
- SM_MODEL_DIR: A string representing the path to the directory to write model artifacts to. These artifacts are uploaded to S3 for model hosting.
- SM_OUTPUT_DATA_DIR: A string representing the filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.
- Supposing two input channels, ‘train’ and ‘test’, were used in the call to the Scikit-learn estimator’s fit() method, the following will be set, following the format “SM_CHANNEL_[channel_name]”:
- SM_CHANNEL_TRAIN: A string representing the path to the directory containing data in the ‘train’ channel
- SM_CHANNEL_TEST: Same as above, but for the ‘test’ channel.

In order to save your trained Scikit-learn model for deployment on SageMaker, your training script should save your model to a certain filesystem path called model_dir. This value is accessible through the environment variable SM_MODEL_DIR.

Load the model: before a model can be served, it must be loaded. The SageMaker Scikit-learn model server loads your model by invoking a 'model_fn' function that you must provide in your script

Serve a Model: after the SageMaker model server has loaded your model by calling model_fn, SageMaker will serve your model. Model serving is the process of responding to inference requests, received by SageMaker InvokeEndpoint API calls. The SageMaker Scikit-learn model server breaks request handling into three steps:

-input processing,

-prediction, and

-output processing.

## define the estimator that takes your training script

In [ ]:
sklearn_estimator = SKLearn(entry_point='main.py',
                            instance_type='ml.m4.xlarge',
                            framework_version='0.20.0',
                            role=role)


# Calling `fit`
To start a training job, we call `estimator.fit(training_data_uri)`.

An S3 location is used here as the input. fit creates a default channel named 'training', which points to this S3 location. In the training script we can then access the training data from the location stored in SM_CHANNEL_TRAINING. fit accepts a couple other types of input as well. See the API doc [here](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) for details.

When training starts, the Scikit-learn container executes BYO_sklearn_main.py, passing hyperparameters and model_dir from the estimator as script arguments. Because we didn't define either in this example, no hyperparameters are passed, and model_dir defaults to `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`, so the script execution is as follows:

`BYO_sklearn_main.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`

When training is complete, the training job will upload the saved model to S3 for deployment.

In [ ]:
sklearn_estimator.fit({'train': 's3://{}/data/data.csv'.format(bucket)})


## Deploy
We are now ready to deploy our model to Sagemaker hosting services and make real time predictions

In [ ]:
predictor = sklearn_estimator.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1)


Let's now send some data to our model to predict- the data shouldbe sent in the accepted format and the code below just does that.

In [ ]:
from sklearn.preprocessing import StandardScaler
test=pd.read_csv("data.csv")
#print(test)
test=test.iloc[1:50,1:5].values.tolist()
#standardised input
sc = StandardScaler()
test = sc.fit_transform(test)

    
test

In [ ]:
request_body = ""
for row in test:
    request_body += ",".join([str(n) for n in row]) + "\n"
request_body = request_body[:-1]

print(request_body)

In [ ]:
client = boto3.client('sagemaker-runtime')

endpoint=predictor.endpoint_name


content_type = "text/csv"

response = client.invoke_endpoint(
    EndpointName=endpoint,
    ContentType=content_type,
    Body=request_body
    )


In [ ]:
response['Body'].read()